In [1]:
from sqlalchemy import create_engine, Column, Integer, String, DateTime, Text, PrimaryKeyConstraint
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from dotenv import load_dotenv
import os
from datetime import datetime
import random
from sqlalchemy.dialects.postgresql import INET  # Для IP-адресов

In [2]:
load_dotenv()

True

In [3]:
engine = create_engine(
    f"postgresql+psycopg2://{os.getenv('POSTGRES_USER')}:{os.getenv('POSTGRES_PASSWORD')}@"
    f"{os.getenv('POSTGRES_HOST')}:{os.getenv('POSTGRES_PORT')}/{os.getenv('POSTGRES_DB')}"
)

In [4]:
Base = declarative_base()

class WebLog(Base):
    __tablename__ = 'web_logs'
    
    id = Column(Integer, primary_key=True)
    ip = Column(INET())  # Оптимальный тип для IP (вместо VARCHAR)
    timestamp = Column(DateTime(timezone=True))  # С временной зоной!
    method = Column(String(10))  # Можно оставить, но лучше ENUM
    url = Column(Text)
    status = Column(Integer)
    size = Column(Integer)
    
    # Важно: timestamp должен быть в первичном ключе
    __table_args__ = (
        PrimaryKeyConstraint('id', 'timestamp'),  # Составной ключ
    )
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)

/tmp/ipykernel_1423/3209646392.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()
/tmp/ipykernel_1423/3209646392.py:3: SAWarning: Table 'web_logs' specifies columns 'id' as primary_key=True, not matching locally specified columns 'id', 'timestamp'; setting the current primary key columns to 'id', 'timestamp'. This warning may become an exception in a future release
  class WebLog(Base):


In [5]:
def generate_log_entry():
    # 1. Генерируем IP (случайный адрес в локальной сети)
    ip = f"192.168.{random.randint(0, 255)}.{random.randint(0, 255)}"
    
    # 2. Форматируем текущее время
    timestamp = datetime.now().strftime('[%Y-%m-%d %H:%M:%S]')
    
    # 3. Выбираем случайные значения для остальных полей
    method = random.choice(["GET", "POST", "PUT", "DELETE"])
    url = random.choice(["/api/data", "/users", "/products"])
    status = random.choice([200, 201, 400, 404, 500])
    size = random.randint(100, 5000)  # Размер от 100 до 5000 байт
    
    # 4. Собираем строку
    return f'{ip} {timestamp} "{method} {url} HTTP/1.1" {status} {size}'

def parse_log_line(log_line):
    """Разбирает строку лога на компоненты."""
    parts = log_line.split()
    return {
        'ip': parts[0],
        'timestamp': datetime.strptime((parts[1] + ' ' + parts[2])[1:-1], '%Y-%m-%d %H:%M:%S'),
        'method': parts[3][1:],  # Убираем первую кавычку
        'url': parts[4],
        'status': int(parts[6]),
        'size': int(parts[7]),
        #'raw_log': log_line
    }

def save_logs_to_db(num_entries=100):
    session = Session()
    
    for _ in range(num_entries):
        log_line = generate_log_entry()
        log_data = parse_log_line(log_line)
        session.add(WebLog(**log_data))
    
    session.commit()
    print(f"Сохранено {num_entries} записей в БД")

# Пример использования
save_logs_to_db(1_000_000)

TypeError: 'raw_log' is an invalid keyword argument for WebLog